In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sc
import math

In [4]:
####################################
#### load in all the gene lists ####
####################################

### edited 14.8.2019 to update to newest SE gene lists (taken from pause-initiation notebook)
### edited 24.10.2019 to also include new control sets

SE_genes = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\SE_464_genes.xlsx')
TF_genes = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\TF_53_genes.xlsx')
CRC_genes = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\autoregulatory_6motifs_24_genes.xlsx')
SE_control_genes = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\control_464_all_2e.xlsx')
TF_control_genes = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\control_53_all_3e.xlsx')
TF_control_onlyTFs = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\control_53_TFs_3f.xlsx')
CRC_control_genes = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\control_24_all_4e.xlsx')
CRC_control_onlyTFs = pd.read_excel('S:\\lab_winter\\martin\\MED14_PRO-seq_final\\other_final_files\\SE_and_control_gene_sets\\KBM7\\control_24_TFs_4f.xlsx')

In [5]:
######################################
#### grep genes from DEseq tables ####
######################################
# DEseq2 dataframe
DEseq_TT_1h_df = pd.read_csv('S:\\lab_winter\\martin\\MJ-19-1\\20190418_JaegerWinter2019TTseqKBM7\\DifferentialExpression\\differential.expression.estimates_1h.txt', sep="\t")

##### grep autoreg CRC genes #####
# use query to parse geneIDs from the DEseq result table. Need to use the '|'.join construct to use geneIDs as REGEX, where some TRxxx annotations have more than on gene_id
CRC_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(CRC_genes.geneID.tolist()), na=False)]

##### same for 53 TF SE-genes #####
TF_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(TF_genes.geneID.tolist()), na=False)]

##### same for 464 SE genes #####
SE_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(SE_genes.geneID.tolist()), na=False)]

##### same for 464 control genes #####
SE_control_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(SE_control_genes.geneID.tolist()), na=False)]

##### same for 53 control genes #####
TF_control_genes_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(TF_control_genes.geneID.tolist()), na=False)]

##### same for 53 control TFs #####
TF_control_onlyTFs_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(TF_control_onlyTFs.geneID.tolist()), na=False)]

##### same for 24 control genes #####
CRC_control_genes_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(CRC_control_genes.geneID.tolist()), na=False)]

##### same for 24 control TFs #####
CRC_control_onlyTFs_DEseq = DEseq_TT_1h_df[DEseq_TT_1h_df.gene_id.str.contains('|'.join(CRC_control_onlyTFs.geneID.tolist()), na=False)]


In [6]:
##############################################
#### generate CRC network cytoscape input ####
##############################################
# first the node interactions as SIF format (columns: source | interaction | target | no_of_motifs):
motif_df = pd.read_csv('S:\\lab_winter\\martin\\Mediator_CRC\\Number_of_motifs_24autoregulatoryTFs.txt', sep="\t")
motif_df.Number_of_SEs = "pp"

#motif_df.to_csv('S:\\lab_winter\\martin\\Mediator_CRC\\Number_of_motifs_24autoregulatoryTFs.sif', sep="\t", header=False)

# now node attributes:
CRC_annotated = pd.DataFrame(columns=CRC_DEseq.columns)
for gene in CRC_genes.geneID: #["ENSG00000070476", "ENSG00000136997"]:
    row = CRC_DEseq[CRC_DEseq.gene_id.str.contains(gene)]
    row.loc[row.index, 'single_ID'] = gene
    row.loc[row.index, 'gene_name'] = CRC_genes.loc[CRC_genes.geneID.str.contains(gene), 'gene_name'].values[0]
    CRC_annotated = CRC_annotated.append(row, sort=False)
    
CRC_annotated.to_csv('S:\\lab_winter\\martin\\Mediator_CRC\\networks\\CRC_22_network_node_attributes_final.txt', sep="\t")

C:\Users\mjaeger\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\mjaeger\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ValueError: cannot set a frame with no defined index and a scalar

In [63]:
DEseq_TT_1h_df.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,1.955900e+04,1.955900e+04,1.955900e+04,19559.000000,19559.000000,19559.000000,19559.000000,1.955900e+04,1.369100e+04
mean,7.422620e+07,7.429225e+07,6.604704e+04,3431.298782,-0.298344,0.226941,-1.996209,2.929403e-01,1.700357e-01
std,5.514831e+07,5.515504e+07,1.252735e+05,8847.364556,0.356274,0.126891,2.348907,3.369512e-01,2.618431e-01
min,1.187400e+04,1.440900e+04,7.300000e+01,0.250000,-3.766876,0.086348,-32.225623,7.725228e-228,1.057661e-223
25%,3.181257e+07,3.188304e+07,1.024600e+04,10.250000,-0.470841,0.119058,-3.289508,9.385812e-04,3.127187e-04
50%,6.075623e+07,6.086803e+07,2.713400e+04,385.000000,-0.289681,0.174763,-1.497543,1.198924e-01,2.081419e-02
75%,1.100451e+08,1.101722e+08,6.821700e+04,3001.687500,-0.078162,0.321369,-0.300253,5.791104e-01,2.482001e-01
max,2.489062e+08,2.489191e+08,2.304636e+06,217312.500000,1.601821,0.484902,10.812809,1.000000e+00,1.000000e+00


In [65]:
TF_DEseq.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,5.300000e+01,5.300000e+01,53.000000,53.000000,53.000000,53.000000,53.000000,5.300000e+01,5.300000e+01
mean,7.375180e+07,7.387574e+07,123948.773585,25820.393868,-1.099730,0.133880,-8.846924,1.601910e-02,2.107550e-02
std,5.364301e+07,5.363964e+07,134305.272616,32531.891272,0.800850,0.060335,6.163941,6.118565e-02,7.693931e-02
min,2.716210e+05,4.382900e+05,2471.000000,32.250000,-3.766876,0.088807,-32.225623,7.725228e-228,1.057661e-223
25%,3.683328e+07,3.703448e+07,18461.000000,3029.000000,-1.308204,0.101307,-11.171217,5.640257e-29,6.227480e-27
50%,5.429211e+07,5.430104e+07,87454.000000,7230.375000,-0.966589,0.111961,-7.614272,2.651821e-14,8.603338e-13
75%,1.048541e+08,1.048586e+08,166670.000000,42111.375000,-0.534698,0.145144,-4.015698,5.927020e-05,2.475498e-04
max,2.129997e+08,2.131516e+08,629276.000000,125676.875000,-0.109299,0.414080,-0.972777,3.306640e-01,4.025180e-01


In [66]:
CRC_DEseq.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,2.400000e+01,2.400000e+01,24.000000,24.000000,24.000000,24.000000,24.000000,2.400000e+01,2.400000e+01
mean,6.543511e+07,6.553447e+07,99355.375000,26186.432292,-1.392394,0.117455,-11.620007,2.899057e-03,4.632685e-03
std,5.338847e+07,5.338477e+07,93191.424738,32253.578102,0.969435,0.027770,7.122732,1.355342e-02,2.108655e-02
min,4.979412e+06,4.985180e+06,5366.000000,429.125000,-3.766876,0.091020,-32.225623,7.725228e-228,1.057661e-223
25%,3.212255e+07,3.218557e+07,34324.000000,3774.281250,-1.589246,0.100861,-14.030235,9.935547e-42,2.429064e-39
50%,4.483409e+07,4.488354e+07,81312.500000,8208.500000,-1.161248,0.109033,-9.957587,1.088748e-21,7.236215e-20
75%,1.012901e+08,1.013005e+08,131145.750000,43352.406250,-0.744533,0.126206,-7.324853,2.564404e-13,7.164011e-12
max,2.129997e+08,2.131516e+08,369898.000000,125676.875000,-0.186592,0.219356,-1.835209,6.647477e-02,1.034212e-01


In [6]:
SE_control_DEseq.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,4.390000e+02,4.390000e+02,439.000000,439.000000,439.000000,439.000000,439.000000,4.390000e+02,4.350000e+02
mean,7.701726e+07,7.707567e+07,58410.070615,6639.015661,-0.402854,0.125609,-3.353318,6.406750e-02,7.589038e-02
std,5.807262e+07,5.807646e+07,67871.604887,8288.023154,0.241148,0.039116,1.915337,1.552390e-01,1.672120e-01
min,1.244760e+05,2.023060e+05,1176.000000,0.250000,-1.550282,0.086907,-12.792303,1.810289e-37,3.442316e-35
25%,3.328805e+07,3.332018e+07,15322.000000,1527.437500,-0.491830,0.103157,-4.203086,2.634141e-05,1.166813e-04
50%,5.765774e+07,5.766186e+07,34619.000000,3593.000000,-0.371171,0.112272,-3.126367,1.769806e-03,4.356602e-03
75%,1.160582e+08,1.160971e+08,74854.000000,7848.750000,-0.266774,0.133117,-2.127717,3.337346e-02,5.299628e-02
max,2.475071e+08,2.475777e+08,415247.000000,56691.500000,0.194954,0.426372,1.182210,9.726090e-01,9.783256e-01


In [7]:
TF_control_genes_DEseq.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,5.200000e+01,5.200000e+01,52.000000,52.000000,52.000000,52.000000,52.000000,5.200000e+01,5.200000e+01
mean,7.561245e+07,7.566525e+07,52796.384615,8114.882212,-0.456175,0.118141,-4.109765,3.310812e-02,4.158777e-02
std,5.552803e+07,5.553357e+07,65429.283733,10553.548709,0.318525,0.029270,2.488845,1.182043e-01,1.295526e-01
min,4.847631e+06,4.882401e+06,2468.000000,148.875000,-1.546329,0.092298,-13.297728,2.386087e-40,5.444653e-38
25%,3.278481e+07,3.282275e+07,20541.500000,2127.593750,-0.522642,0.101787,-5.157603,2.860720e-07,2.371383e-06
50%,5.686734e+07,5.692414e+07,31435.000000,4799.250000,-0.408366,0.109857,-3.750081,1.846614e-04,6.515500e-04
75%,1.147077e+08,1.147758e+08,56907.000000,8669.218750,-0.323135,0.116658,-2.810960,4.969868e-03,1.097927e-02
max,2.243851e+08,2.244343e+08,333894.000000,53267.500000,0.531192,0.243378,2.182579,7.867894e-01,8.228503e-01


In [8]:
TF_control_onlyTFs_DEseq.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,5.300000e+01,5.300000e+01,53.000000,53.000000,53.000000,53.000000,53.000000,5.300000e+01,5.300000e+01
mean,7.002064e+07,7.008794e+07,67306.830189,10366.240566,-0.479180,0.120347,-4.027338,2.900520e-02,3.789304e-02
std,4.949344e+07,4.950989e+07,74004.319607,12639.790728,0.299096,0.027411,2.020253,9.304635e-02,1.090104e-01
min,1.244760e+05,2.023060e+05,2321.000000,372.750000,-2.083259,0.087686,-11.646339,2.395288e-31,3.183873e-29
25%,4.016137e+07,4.024092e+07,18502.000000,2529.500000,-0.517263,0.102762,-4.494719,6.966176e-06,3.936191e-05
50%,6.088150e+07,6.092816e+07,37122.000000,5447.750000,-0.440701,0.112034,-3.957652,7.569013e-05,3.035365e-04
75%,9.961695e+07,9.963241e+07,90732.000000,13801.375000,-0.331929,0.126651,-2.792016,5.238070e-03,1.148349e-02
max,2.075299e+08,2.076056e+08,297297.000000,52267.625000,-0.072286,0.205724,-0.648745,5.165032e-01,5.822995e-01


In [9]:
CRC_control_genes_DEseq.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,2.400000e+01,2.400000e+01,24.000000,24.000000,24.000000,24.000000,24.000000,2.400000e+01,23.000000
mean,7.283632e+07,7.287349e+07,37174.500000,6466.364583,-0.390921,0.134775,-3.200046,4.452301e-02,0.038318
std,6.174076e+07,6.172927e+07,60732.150349,12338.233267,0.121484,0.056312,1.247367,1.033233e-01,0.068303
min,2.145200e+05,2.680590e+05,1702.000000,2.000000,-0.593991,0.100423,-4.914561,8.898147e-07,0.000007
25%,2.605423e+07,2.606095e+07,7461.250000,1101.031250,-0.482538,0.107601,-4.361217,1.313153e-05,0.000062
50%,6.253791e+07,6.255557e+07,16663.500000,2947.625000,-0.381437,0.112793,-3.509570,4.693702e-04,0.001065
75%,1.111931e+08,1.112260e+08,39773.250000,5818.968750,-0.289107,0.133574,-1.974822,4.850287e-02,0.044963
max,2.351312e+08,2.351613e+08,281392.000000,59205.375000,-0.190801,0.373622,-0.715648,4.742088e-01,0.235149


In [10]:
CRC_control_onlyTFs_DEseq.describe()

,start,end,length,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
count,2.400000e+01,2.400000e+01,24.000000,24.000000,24.000000,24.000000,24.000000,2.400000e+01,2.400000e+01
mean,7.431172e+07,7.438324e+07,71526.500000,11413.557292,-0.525314,0.115618,-4.317213,1.354995e-02,1.948430e-02
std,6.059437e+07,6.059928e+07,92712.697572,16858.664687,0.380344,0.022417,2.137031,4.879388e-02,6.287040e-02
min,5.586999e+06,5.622927e+06,4505.000000,719.625000,-1.960291,0.088233,-10.304175,6.746814e-25,5.883479e-23
25%,2.557921e+07,2.567524e+07,14468.750000,2579.375000,-0.535300,0.102619,-4.616061,4.087655e-06,2.489287e-05
50%,5.294907e+07,5.296333e+07,31633.000000,5026.500000,-0.448454,0.110709,-4.006551,6.286409e-05,2.586770e-04
75%,1.200888e+08,1.201035e+08,90779.750000,11244.218750,-0.356167,0.117020,-3.128231,1.799064e-03,4.585131e-03
max,2.208794e+08,2.208851e+08,342810.000000,76970.125000,-0.120425,0.190242,-1.178411,2.386329e-01,3.055144e-01


In [15]:
np.log10(CRC_control_onlyTFs_DEseq.padj.median())

-3.587242257197747